In [1]:
import torch  # 这就是你所需要用PyTorch和TorchScript的所有。
print(torch.__version__)

1.5.1


In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()

    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[0.5902, 0.7705, 0.5580, 0.6256],
        [0.8927, 0.7611, 0.8371, 0.4690],
        [0.3907, 0.9546, 0.5320, 0.6729]]), tensor([[0.5902, 0.7705, 0.5580, 0.6256],
        [0.8927, 0.7611, 0.8371, 0.4690],
        [0.3907, 0.9546, 0.5320, 0.6729]]))


In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.6463,  0.1571, -0.2479,  0.4087],
        [ 0.7607,  0.6523,  0.7923,  0.3044],
        [ 0.2479,  0.7824,  0.0745,  0.3718]], grad_fn=<TanhBackward>), tensor([[ 0.6463,  0.1571, -0.2479,  0.4087],
        [ 0.7607,  0.6523,  0.7923,  0.3044],
        [ 0.2479,  0.7824,  0.0745,  0.3718]], grad_fn=<TanhBackward>))


In [4]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.9448, -0.0326,  0.2476,  0.7876],
        [ 0.9324,  0.7594,  0.7068,  0.5698],
        [ 0.8284,  0.7522,  0.5473,  0.7250]], grad_fn=<TanhBackward>), tensor([[ 0.9448, -0.0326,  0.2476,  0.7876],
        [ 0.9324,  0.7594,  0.7068,  0.5698],
        [ 0.8284,  0.7522,  0.5473,  0.7250]], grad_fn=<TanhBackward>))


In [5]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[ 0.7917, -0.0382,  0.2614,  0.8160],
         [ 0.5807,  0.6550,  0.6552,  0.9022],
         [ 0.4418,  0.6988,  0.8347,  0.7276]], grad_fn=<TanhBackward>),
 tensor([[ 0.7917, -0.0382,  0.2614,  0.8160],
         [ 0.5807,  0.6550,  0.6552,  0.9022],
         [ 0.4418,  0.6988,  0.8347,  0.7276]], grad_fn=<TanhBackward>))

In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %input : Float(3, 4),
      %h : Float(3, 4)):
  %19 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %21 : Tensor = prim::CallMethod[name="forward"](%19, %input)
  %12 : int = prim::Constant[value=1]() # <ipython-input-5-1f6e08af67d0>:7:0
  %13 : Float(3, 4) = aten::add(%21, %h, %12) # <ipython-input-5-1f6e08af67d0>:7:0
  %14 : Float(3, 4) = aten::tanh(%13) # <ipython-input-5-1f6e08af67d0>:7:0
  %15 : (Float(3, 4), Float(3, 4)) = prim::TupleConstruct(%14, %14)
  return (%15)



In [7]:
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = torch.add((self.linear).forward(input, ), h, alpha=1)
  _1 = torch.tanh(_0)
  return (_1, _1)



In [8]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[ 0.7917, -0.0382,  0.2614,  0.8160],
        [ 0.5807,  0.6550,  0.6552,  0.9022],
        [ 0.4418,  0.6988,  0.8347,  0.7276]], grad_fn=<TanhBackward>), tensor([[ 0.7917, -0.0382,  0.2614,  0.8160],
        [ 0.5807,  0.6550,  0.6552,  0.9022],
        [ 0.4418,  0.6988,  0.8347,  0.7276]], grad_fn=<TanhBackward>))
(tensor([[ 0.7917, -0.0382,  0.2614,  0.8160],
        [ 0.5807,  0.6550,  0.6552,  0.9022],
        [ 0.4418,  0.6988,  0.8347,  0.7276]], grad_fn=<TanhBackward>), tensor([[ 0.7917, -0.0382,  0.2614,  0.8160],
        [ 0.5807,  0.6550,  0.6552,  0.9022],
        [ 0.4418,  0.6988,  0.8347,  0.7276]], grad_fn=<TanhBackward>))


In [9]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(input, ), )
  _1 = torch.tanh(torch.add(_0, h, alpha=1))
  return (_1, _1)



In [10]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
traced_cell = torch.jit.script(my_cell)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  new_h = torch.tanh(torch.add(_0, h, alpha=1))
  return (new_h, new_h)



In [11]:
# 新的输入
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

(tensor([[0.5520, 0.9338, 0.5911, 0.5728],
         [0.8312, 0.5042, 0.6719, 0.3194],
         [0.2192, 0.4774, 0.2456, 0.2004]], grad_fn=<TanhBackward>),
 tensor([[0.5520, 0.9338, 0.5911, 0.5728],
         [0.8312, 0.5042, 0.6719, 0.3194],
         [0.2192, 0.4774, 0.2456, 0.2004]], grad_fn=<TanhBackward>))

In [12]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    _0 = (self.cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [13]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(argument_1, )
  return torch.relu(y)



In [14]:
traced.save('wrapped_rnn.zip')

loaded = torch.jit.load('wrapped_rnn.zip')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(argument_1, )
  return torch.relu(y)

